In [2]:
import os
import cv2
import numpy as np
from skimage.filters import frangi
from tqdm import tqdm

ROI_DIR = r"roi"
MASK_DIR = r"masks"

os.makedirs(MASK_DIR, exist_ok=True)

roi_images = [f for f in os.listdir(ROI_DIR)
              if f.lower().endswith((".png", ".jpg", ".jpeg"))]

for fname in tqdm(roi_images):
    img_path = os.path.join(ROI_DIR, fname)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    if img is None:
        continue

    # --- Step 1: Denoise ---
    img_blur = cv2.GaussianBlur(img, (5, 5), 0)

    # --- Step 2: Vessel enhancement ---
    vessel = frangi(img_blur)
    vessel = (vessel / vessel.max() * 255).astype(np.uint8)

    # --- Step 3: Threshold ---
    _, binary = cv2.threshold(
        vessel, 0, 255,
        cv2.THRESH_BINARY + cv2.THRESH_OTSU
    )

    # --- Step 4: Morphology cleanup ---
    kernel = np.ones((3, 3), np.uint8)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=2)
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=1)

    # --- Step 5: Keep largest connected component ---
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary)

    if num_labels > 1:
        largest = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
        mask = np.zeros_like(binary)
        mask[labels == largest] = 255
    else:
        mask = binary

    # --- Save ---
    mask_name = os.path.splitext(fname)[0] + "_mask.png"
    cv2.imwrite(os.path.join(MASK_DIR, mask_name), mask)

print("Automatic lumen masks generated.")


100%|██████████| 1233/1233 [03:20<00:00,  6.14it/s]

Automatic lumen masks generated.
